### fast weighted random sampling (python)
- w=Weight_Random_Sample(D) # initialize a sampler with a dictionary object whose keys are to be sampled and the values are the weights of the keys. 
- w.spl(n) # generate a sample of population n.

In [2]:
import numpy as np
import time
from random import sample as rand_sample

In [3]:
class Weight_Random_Sample:
    def __init__(self,Dict):
        self.probs =[]
        self.keys =[]
        nk = 0
        for k in Dict:
            self.probs.append(Dict[k])
            self.keys[nk] =k
            nk += 1
        self.probs = np.array(self.probs, dtype=float)
        self.probs = self.probs/ self.probs.sum()
        self.setup()
    def setup(self):
        K = len(self.probs)
        q = np.zeros(K)
        J = np.zeros(K, dtype = int)
        
        # sort the data into the outcomes with probabilities
        # that are larger and smaller than 1/K
        smaller =[]
        larger =[]
        for kk, prob in enumerate(self.probs):
            q[kk] = K*prob
            if q[kk] <1.0:
                smaller.append(kk)
            else:
                larger.append(kk)
                
        # Loop through and create little binary mixtures that 
        # appropriately allocate the larger outcomes over the overall uniform mixture
        while len(smaller) >0 and len(larger) >0:
            samll = smaller.pop
            large = larger.pop
            
            J[small] = large
            q[large] =q[large] + q[samll]-1.0
            
            if q[large] <1:
                smaller.append(large)
            else:
                larger.append(large)
                
        self.J = J
        self.q = q
    
    def draw(self):
        K = len(self.J)
        
        # Draw from the overall uniform mixture
        kk = int(floor(random.rand()*K))
        
        # Draw from the binary mixture, either keeping the small one,
        # or choosing the associated larger one.
        if random.rand() < self.q[kk]:
            return self.keys[kk]
        else:
            return self.keys[self.J[kk]]
        
    def spl(self, n, timing =False, nopref=False):
        if timing:
            tic = time.time()
        # Generate variates
        if nopref:
            X = rand_sample(self.keys.values(),n)
        else:
            X = []
            for nn in xrange(n):
                X.append(self.draw())
                
        if timing:
            print("Time elapsed:"+str(time.time()-tic))
        return X